In [ ]:
# merge_consec_bookings(initial/BookingMultiDestinations) -> 'processed/my-df-mdt'

In [ ]:
# from Modules.data_processing import set_rewards_toppop


# set_rewards_toppop('data/processed/my-df-mdt.csv', save_name='data/processed/my-pop-mdt-buy.csv', pop_ratio=0.05)
# set_rewards_toppop('data/processed/test-my-df-mdt.csv', save_name='data/processed/test-my-pop-mdt-buy.csv', pop_ratio=0.05)

In [ ]:
import pandas as pd
import tensorflow as tf
import os

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# dat = '/content/drive/MyDrive/BOOKING/'  # for collab
dat = 'data/processed/'

In [ ]:
from Modules.data_processing import preprocess_booking, get_statistics, create_pop_dict

In [ ]:
df_train, itm2idx, idx2itm = preprocess_booking(dat + 'my-time-mdt-buy.csv', save_name='sorted_travels.csv')
df_test, _, idx2itm  = preprocess_booking(dat + 'test-my-time-mdt-buy.csv', save_name='test_sorted_travels.csv', itm2idx=itm2idx)

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
df = df.sort_values(by=['session_id', 'timestamp'])

In [ ]:
import matplotlib.pyplot as plt


plt.hist(df_train.session_id.value_counts())

In [ ]:
#replay_buf, data_stats = get_statistics(df, df_train, length=10)
replay_buf, data_stats = pd.read_pickle(dat + 'replay_buffer.df'), pd.read_pickle(dat + 'data_stats.df')

In [ ]:
#create_pop_dict(data_dir='data/processed/', filename='sorted_travels.csv')

In [ ]:
from Modules.training import train_no_eval, predict_booking_batched

In [ ]:
arg_dict = {'r_click' : 0.7,
            'r_buy' : 1,
            'r_negative' : 1,
            'hidden_factor' : 64,
            'lr' : 0.015,
            'lr2' : 0.001,
            'epoch' : 20,
            'batch_size' : 512,
            'neg' : 5,
            'discount' : 0.7,
            'smooth' : 0.0,
            'clip' : 0.0
            }
losses_sa2c = []
SA2C_1, sess_sa2c  = train_no_eval(data_stats, replay_buf, arg_dict=arg_dict, losses=losses_sa2c, configuration='SA2C', sa2c_switch_step=10000, data_dir='.')

In [ ]:
ground_truth = pd.read_csv(dat + 'ground_truth.csv').drop(['id'], axis=1)
ground_truth['city_id'] = ground_truth['city_id'].map(itm2idx)

In [ ]:
predictions = predict_booking_batched(sess_sa2c, SA2C_1, df_test, data_stats)

In [ ]:
predictions

In [ ]:
def evaluate_accuracy_at(submission,ground_truth, at=4):
    '''checks if the true city is within the four recommended cities'''
    data = submission.merge(ground_truth,on='utrip_id')

    if at==4:
      hits = ((data['city_id']==data['city_id_1'])|(data['city_id']==data['city_id_2'])|
        (data['city_id']==data['city_id_3'])|(data['city_id']==data['city_id_4']))*1
    elif at==10:
      hits = ((data['city_id']==data['city_id_1'])|(data['city_id']==data['city_id_2'])|
        (data['city_id']==data['city_id_3'])|(data['city_id']==data['city_id_4'])|
        (data['city_id']==data['city_id_5'])|(data['city_id']==data['city_id_6'])|
        (data['city_id']==data['city_id_7'])|(data['city_id']==data['city_id_8'])|
        (data['city_id']==data['city_id_9'])|(data['city_id']==data['city_id_10']))*1
    return hits.mean()

In [ ]:
evaluate_accuracy_at(predictions,ground_truth, at=4)

In [ ]:
topcities = df_train.item_id.value_counts().index[:10]

In [ ]:
test_trips = (df_test[['session_id']].drop_duplicates()).reset_index().drop('index', axis=1)
cities_prediction = pd.DataFrame([topcities]*test_trips.shape[0]
                                 , columns= [f'city_id_{i+1}' for i in range(10)])

In [ ]:
submission = pd.concat([test_trips,cities_prediction], axis =1)

In [ ]:
submission = submission.rename({'session_id':'utrip_id'}, axis=1)

In [ ]:
submission

In [ ]:
evaluate_accuracy_at(submission, ground_truth, at=4)